### This notebook reproduces the visualization from our technical report of the datasets returned by Auctus for the search query "taxi" on May 4th, 2023. To see the full pipeline or produce a visualization for the results of another search query, please see https://github.com/egm68/dataset-visualization/blob/main/full_pipeline.ipynb.

First we install and import the required libraries and functions. The final two imports bring in functions from two other files in this repository, netgraph_functions.py and custom_functions.py. 

The file netgraph_functions.py contains a snapshot of a selection of functions from netgraph, a python library created by Paul Broderson which can be found at https://github.com/paulbrodersen/netgraph. We preserve these functions in a static file for reproducability purposes, since the functions we use were added to the library very recently and the library is edited frequently. 

The file custom_functions.py contains functions written by the authors of this repository for this project. 

In [ ]:
#installations
!pip install kshingle

In [ ]:
!pip install prince

In [ ]:
!pip install rectangle-packer

In [ ]:
!pip install grandalf

In [1]:
#imports
import pandas as pd
import numpy as np
from datetime import datetime
import kshingle as ks
import random
import math
import itertools
import prince
import warnings
from functools import wraps
from itertools import combinations, product
from scipy.spatial import Voronoi
from scipy.spatial.distance import cdist, pdist, squareform
from scipy.optimize import minimize, NonlinearConstraint
from rpack import pack
from grandalf.graphs import Vertex, Edge, Graph
from grandalf.layouts import SugiyamaLayout, DummyVertex
from netgraph_functions import get_geometric_layout, _initialise_geometric_node_layout, _flatten, _get_unique_nodes, get_fruchterman_reingold_layout, _edge_list_to_adjacency_matrix, _get_fr_repulsion, _get_fr_attraction, _fruchterman_reingold, _get_temperature_decay, _is_within_bbox, _fit_to_frame, _get_angle, _rotate
from custom_functions import jaccard_similarity, distance_from_similarity, get_edges, get_edge_lengths, normalize, get_node_positions, get_df_cols

Next we load the metadata for the datasets returned by Auctus for the search query "taxi" on May 4th, 2023. 

In [2]:
#load metadata dataframe from local directory
df = pd.read_csv("taxi_metadata_2023_05_04.csv")
display(df)

,title,description,size,num_spatial,num_categorical,num_temporal,num_columns,start_date,end_date,temporal_col_names,cat_col_names,spatial_col_names,all_col_names,full_metadata,date_of_search
0,Taxi Medallion Transfers,Operation of a taxi cab in Chicago requires a ...,304895,0,0,1,5,2007-10-10 00:00:00.000000,2017-07-21 00:00:00.000000,closing_date,NaN,NaN,"closing_date, public_vehicle_number, sale_pric...","{'name': 'Taxi Medallion Transfers', 'source':...",2023-05-04 14:05:57.777532
1,2017 Yellow Taxi Data,This dataset includes trip records from all tr...,13949149,0,0,1,3,2017-07-03 23:00:16.000000,2017-11-04 03:58:56.000000,tpep_pickup_datetime,tpep_pickup_datetime,NaN,"tpep_pickup_datetime, pulocationid, n._trips",{'attribute_keywords': ['tpep_pickup_datetime'...,2023-05-04 14:05:57.777565
2,Green Taxi Data 2015,This dataset contains green taxi trip records ...,1066116963,4,1,2,23,2015-01-02 21:26:24.000000,2015-03-23 08:42:40.000000,"pickup_datetime, dropoff_datetime",store_and_fwd_flag,"pickup_longitude, pickup_latitude, dropoff_lon...","vendorid, pickup_datetime, dropoff_datetime, s...","{'attribute_keywords': ['VendorID', 'Vendor', ...",2023-05-04 14:05:57.777682
3,Yellow Taxi Data 2015,This dataset contains the daily number of yell...,20244,0,0,1,4,2015-01-07 00:00:00.000000,2015-09-05 00:00:00.000000,pickup_datetime,NaN,NaN,"pickup_datetime, n._trips, price, distance","{'attribute_keywords': ['pickup_datetime', 'pi...",2023-05-04 14:05:57.777710
4,2017-yellow-cab-lga,"new york, taxi, yellow cab, LaGuardia, LGA, ci...",319452961,0,1,2,18,2017-01-08 13:20:00.000000,2017-09-04 00:10:40.000000,"tpep_pickup_datetime, tpep_dropoff_datetime",store_and_fwd_flag,NaN,"tpep_pickup_datetime, unnamed:_0, dolocationid...",{'attribute_keywords': ['tpep_pickup_datetime'...,2023-05-04 14:05:57.777785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Monthly Transportation Statistics,Monthly Transportation Statistics is a compila...,492318,0,0,1,136,1948-04-01 00:00:00.000000,1999-05-01 00:00:00.000000,date,NaN,NaN,"index, date, air_safety_-_general_aviation_fat...","{'name': 'Monthly Transportation Statistics', ...",2023-05-04 14:05:57.792827
105,2010 Census/ACS Detailed Block Group Data,detailed characteristics of people and housing...,506671,1,0,0,190,1948-04-01 00:00:00.000000,1999-05-01 00:00:00.000000,NaN,county,NaN,"county, census_tract, block_group, block_group...",{'name': '2010 Census/ACS Detailed Block Group...,2023-05-04 14:05:57.793680
106,2013-2017 American Community Survey Detailed C...,DETAILED CHARACTERISTICS OF PEOPLE AND HOUSING...,435842,0,1,0,211,1948-04-01 00:00:00.000000,1999-05-01 00:00:00.000000,NaN,inside_kcmo_or_not,NaN,"island, census_tract, inside_kcmo_or_not, east...",{'name': '2013-2017 American Community Survey ...,2023-05-04 14:05:57.794625
107,Parking - Edmonton Insight Community,This was one single topic among many as part o...,1085353,0,28,2,74,2016-02-09 13:34:56.000000,2016-02-11 16:34:08.000000,"response_date, completion_date",q4a_parking_it_is_easy_to_find_a_place_to_park...,NaN,"response_date, completion_date, q1a_parking_un...","{'attribute_keywords': ['Response Date', 'Resp...",2023-05-04 14:05:57.795305


We can now calculate the similarity between each pair of datasets based on various attributes. For each of these attributes, we create a set of k-shingles for each dataset and compute the jaccard similarity between each pair of k-shingle sets.

The function jaccard_similarity accepts three arguments, and can be used to calculate the similarity between pairs of datasets using any column containing strings from the metadata dataframe. The first argument is the metadata dataframe. The second argument is a list of the dataset attributes (or metadata dataframe columns) that we will compare each dataset by. These attributes will be concatenated into a single string for each dataset before similarity is calculated. The third argument is k value we will use to create the k-shingle sets for each dataset.  

We will calculate the similarity between each pair of datasets based on their titles, descriptions, and a string containing both their title and description. We choose to break the titles into sets of 5-shingles (since they are relatively short) and the descriptions and title/description strings into sets of 9-shingles (since these are relatively long).  

In [3]:
#get similarity matrices
matrix_title_sim = jaccard_similarity(df, ["title"], 5) 
matrix_desc_sim = jaccard_similarity(df, ["description"], 9) 
matrix_both_sim = jaccard_similarity(df, ["title","description"], 9) 

Our final visualization will contain clusters of dots, each representing one dataset, with the distance between each pair of dots intuitively representing how similar they are. We want dots to be closer together if they are more similar and father apart if they are less similar, and therefore must convert this jaccard similarity to jaccard distance.

In [4]:
#get distance matrices
matrix_title_dist = distance_from_similarity(matrix_title_sim)
matrix_desc_dist = distance_from_similarity(matrix_desc_sim)
matrix_both_dist = distance_from_similarity(matrix_both_sim)

Next we normalize these jaccard distances between the values 0.0001 and 1.0001. This will ensure that the matrices will not break the constraints of the least squares optimization we will perform using these values later (specifically, it will ensure that the matrices do not have a determinant of zero).  

In [5]:
#normalize distances between 0.0001 and 1.0001 rather than 0 and 1 to avoid breaking least squares optimization constraints
matrix_title_dist_norm = normalize(matrix_title_dist, {'actual': {'lower': 0, 'upper': 1}, 'desired': {'lower': 0.0001, 'upper': 1.0001}})
matrix_desc_dist_norm = normalize(matrix_desc_dist, {'actual': {'lower': 0, 'upper': 1}, 'desired': {'lower': 0.0001, 'upper': 1.0001}})
matrix_both_dist_norm = normalize(matrix_both_dist, {'actual': {'lower': 0, 'upper': 1}, 'desired': {'lower': 0.0001, 'upper': 1.0001}})

We can now use the netgraph get_node_positions function to calculate the x and y coordinates of each dataset in the cluster based on the edge lengths (jaccard distances) between them. This function treats the cluster like a force directed graph and uses a sequential least squares programming optimizer to converge on a final layout. The nodes in this graph are datasets, and the length of an edge between two dataset nodes is equal to the jaccard distance between the datasets. These edges will not be visible in the final visualization.

In [6]:
#get edges [tuples in the format (source node ID, target node ID)]  
#and edge lengths [dictionary mapping edges to their distances] for each measure
edges = get_edges(df)

title_edge_lengths = get_edge_lengths(edges, matrix_title_dist_norm)
desc_edge_lengths = get_edge_lengths(edges, matrix_desc_dist_norm)
both_edge_lengths = get_edge_lengths(edges, matrix_both_dist_norm)

In [7]:
#get x and y coordinates for each measure
title_node_positions = get_node_positions(edges, title_edge_lengths)
desc_node_positions = get_node_positions(edges, desc_edge_lengths)
both_node_positions = get_node_positions(edges, both_edge_lengths)

In [8]:
print(title_node_positions[0:10]) #x column and y column 

[[0.69465853 0.10712685]
 [0.88883406 0.52200655]
 [0.16855032 0.49145162]
 [0.59732037 0.23471137]
 [0.32841229 0.20854313]
 [0.6122705  0.42864307]
 [0.58817626 0.41417768]
 [0.370245   0.49514292]
 [0.42301695 0.66694443]
 [0.40473336 0.50949554]]


Next we will use multiple correspondence analysis (MCA) to compute the similarity between the column names within each pair of datasets. Since MCA requires an input that is sorted into multiple categories, we will sort these column names  by whether they represent a "categorical," "spatial," "temporal," or "other" type of variable. 

In [9]:
#separate column names by type
df_cols = get_df_cols(df)
display(df_cols)

,cat_col_names,spatial_col_names,temporal_col_names,misc_col_names
0,,,closing_date,sellers_company_name buyer's_company_name publ...
1,tpep_pickup_datetime,,tpep_pickup_datetime,pulocationid n._trips
2,store_and_fwd_flag,pickup_longitude pickup_latitude dropoff_longi...,pickup_datetime dropoff_datetime,distance passenger_count extra_mta_tax vendori...
3,,,pickup_datetime,distance price n._trips
4,store_and_fwd_flag,,tpep_pickup_datetime tpep_dropoff_datetime,passenger_count vendorid total_amount trip_dis...
...,...,...,...,...
104,,,date,freight_rail_carloads amtrak_on-time_performan...
105,county,,,"residents_16_or_older_employed_in_finance,_ins..."
106,inside_kcmo_or_not,,,"residents_16_or_older_employed_in_finance,_ins..."
107,q4a_parking_it_is_easy_to_find_a_place_to_park...,,response_date completion_date,q1a_parking_buildings_have_awnings_and_overhea...


We can now perform MCA on the dataset column names. 

In [10]:
#perform MCA
X = df_cols
mca = prince.MCA()
mca = mca.fit(X)
mca = mca.transform(X)
display(mca)

,0,1
0,-0.089061,-0.162297
1,-0.095207,-0.230767
2,-0.103479,-0.478016
3,-0.087181,-0.156836
4,-0.098030,-0.385800
...,...,...
104,-0.074821,-0.179583
105,-0.085518,-0.176172
106,-0.084683,-0.183139
107,-0.201614,-0.329811


Earlier we used the netgraph get_node_positions function to find the x and y position of each dataset within each similarity cluster. This function normalizes these positions between 0.05 and 0.95 so that the dots are framed nicely within the 1 by 1 background of the cluster visualization. We therefore normalize the x and y position of each dataset within the column name similarity cluster between 0.05 and 0.95 to match the netgraph padding. 

In [11]:
#normalize values between 0.05 and 0.95
mca_arr = mca.to_numpy()
normalized_mca = np.array(normalize(mca_arr, {'actual': {'lower': mca.min(), 'upper': mca.max()}, 'desired': {'lower': 0.05, 'upper': 0.95}}))

Finally we add the positions of each dataset within each similarity cluster to the dataframe, rename the dataframe index to "id," and save it as a CSV.

In [12]:
#append coordinates to dataframe
df["title_x"] = title_node_positions[:,0]
df["title_y"] = title_node_positions[:,1]

df["description_x"] = desc_node_positions[:,0]
df["description_y"] = desc_node_positions[:,1]

df["title_and_description_x"] = both_node_positions[:,0]
df["title_and_description_y"] = both_node_positions[:,1]

df["column_name_x"] = normalized_mca[:,0]
df["column_name_y"] = normalized_mca[:,1]

df.index.name = 'id'

In [13]:
display(df)

,title,description,size,num_spatial,num_categorical,num_temporal,num_columns,start_date,end_date,temporal_col_names,...,full_metadata,date_of_search,title_x,title_y,description_x,description_y,title_and_description_x,title_and_description_y,column_name_x,column_name_y
id,,,,,,,,,,,,,,,,,,,,,
0,Taxi Medallion Transfers,Operation of a taxi cab in Chicago requires a ...,304895,0,0,1,5,2007-10-10 00:00:00.000000,2017-07-21 00:00:00.000000,closing_date,...,"{'name': 'Taxi Medallion Transfers', 'source':...",2023-05-04 14:05:57.777532,0.694659,0.107127,0.510641,0.480478,0.479707,0.505497,0.065089,0.195021
1,2017 Yellow Taxi Data,This dataset includes trip records from all tr...,13949149,0,0,1,3,2017-07-03 23:00:16.000000,2017-11-04 03:58:56.000000,tpep_pickup_datetime,...,{'attribute_keywords': ['tpep_pickup_datetime'...,2023-05-04 14:05:57.777565,0.888834,0.522007,0.204248,0.791322,0.485572,0.501211,0.064569,0.180172
2,Green Taxi Data 2015,This dataset contains green taxi trip records ...,1066116963,4,1,2,23,2015-01-02 21:26:24.000000,2015-03-23 08:42:40.000000,"pickup_datetime, dropoff_datetime",...,"{'attribute_keywords': ['VendorID', 'Vendor', ...",2023-05-04 14:05:57.777682,0.168550,0.491452,0.510650,0.480477,0.729406,0.911987,0.063871,0.126552
3,Yellow Taxi Data 2015,This dataset contains the daily number of yell...,20244,0,0,1,4,2015-01-07 00:00:00.000000,2015-09-05 00:00:00.000000,pickup_datetime,...,"{'attribute_keywords': ['pickup_datetime', 'pi...",2023-05-04 14:05:57.777710,0.597320,0.234711,0.251418,0.811376,0.784466,0.418800,0.065248,0.196205
4,2017-yellow-cab-lga,"new york, taxi, yellow cab, LaGuardia, LGA, ci...",319452961,0,1,2,18,2017-01-08 13:20:00.000000,2017-09-04 00:10:40.000000,"tpep_pickup_datetime, tpep_dropoff_datetime",...,{'attribute_keywords': ['tpep_pickup_datetime'...,2023-05-04 14:05:57.777785,0.328412,0.208543,0.118997,0.713116,0.912715,0.350613,0.064331,0.146551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Monthly Transportation Statistics,Monthly Transportation Statistics is a compila...,492318,0,0,1,136,1948-04-01 00:00:00.000000,1999-05-01 00:00:00.000000,date,...,"{'name': 'Monthly Transportation Statistics', ...",2023-05-04 14:05:57.792827,0.725267,0.560054,0.510540,0.480409,0.484034,0.503951,0.066292,0.191272
105,2010 Census/ACS Detailed Block Group Data,detailed characteristics of people and housing...,506671,1,0,0,190,1948-04-01 00:00:00.000000,1999-05-01 00:00:00.000000,NaN,...,{'name': '2010 Census/ACS Detailed Block Group...,2023-05-04 14:05:57.793680,0.950000,0.487521,0.701750,0.831516,0.483471,0.501039,0.065388,0.192011
106,2013-2017 American Community Survey Detailed C...,DETAILED CHARACTERISTICS OF PEOPLE AND HOUSING...,435842,0,1,0,211,1948-04-01 00:00:00.000000,1999-05-01 00:00:00.000000,NaN,...,{'name': '2013-2017 American Community Survey ...,2023-05-04 14:05:57.794625,0.527986,0.071833,0.510566,0.480535,0.120733,0.328045,0.065459,0.190500


In [ ]:
df.to_csv("taxi_full_metadata_and_scatterplot_coordinates.csv")